In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import time
from sklearn.externals import joblib
import Recommenders as Recommenders
import Evaluation as Evaluation

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
elec_df = pd.read_csv('/content/drive/My Drive/AIML_NoteBooks/LabAndAssignments/R6/ratings_Electronics.csv',header=None)
elec_df.columns = ["userId", "productId", "ratings", "timestamp"];

elec_df.drop('timestamp', axis=1, inplace=True)
elec_df = elec_df[['userId','productId','ratings']]
elec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824482 entries, 0 to 7824481
Data columns (total 3 columns):
userId       object
productId    object
ratings      float64
dtypes: float64(1), object(2)
memory usage: 179.1+ MB


In [4]:
elec_df.drop_duplicates(subset='userId', keep="last")

,userId,productId,ratings
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0
9,A89DO69P0XZ27,0511189877,5.0
11,A1DA3W4GTFXP6O,0528881469,5.0
12,A29LPQQDG7LD5J,0528881469,1.0
21,A2AW1SSVUIYV9Y,0528881469,4.0
22,A2AEHUKOV014BP,0528881469,5.0
23,AMLFNXUIEMN4T,0528881469,1.0
24,A2O8FIJR9EBU56,0528881469,4.0


In [0]:
# get the number of review by userId
users_by_count = elec_df.groupby('userId')['productId'].count().reset_index(name='counts') 

Selecting products which have more than 50 reviews

In [0]:
top_users = users_by_count[ users_by_count['counts'] > 50]

In [7]:
top_users_reviews = elec_df[elec_df['userId'].isin(top_users['userId'].values)]
top_users_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122171 entries, 118 to 7824444
Data columns (total 3 columns):
userId       122171 non-null object
productId    122171 non-null object
ratings      122171 non-null float64
dtypes: float64(1), object(2)
memory usage: 3.7+ MB


In [8]:
!pip install scikit-surprise

In [9]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(top_users_reviews[['userId', 'productId', 'ratings']], reader)
data.read_ratings

<bound method Dataset.read_ratings of <surprise.dataset.DatasetAutoFolds object at 0x7fa514b762e8>>

In [0]:
splitting data into training and test set for independent attributes

In [0]:
from surprise.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.30)

In [14]:
type(train_data)

surprise.trainset.Trainset

Get the most popular recommmended product

In [15]:
wavg = top_users_reviews.groupby('productId')['ratings'].mean().reset_index(name='avg rating').sort_values('avg rating', ascending=False)
wavg.head(10)

,productId,avg rating
47154,B00LKG1MC8,5.0
18649,B001UE6HZ2,5.0
18704,B001UKJ8FC,5.0
18703,B001UK6UO4,5.0
37782,B0082N9DGY,5.0
37783,B0082N9E9U,5.0
18698,B001UJJNNA,5.0
18695,B001UIPV28,5.0
18694,B001UIHAOK,5.0
37789,B0082PZ0LO,5.0


Use the popularity model to make some predictions

In [16]:
train_data.global_mean

4.2630409616576435

In [17]:
from surprise import SVD, accuracy
algo = SVD()
algo.fit(train_data)

In [0]:
predictions = algo.test(test_data)

In [19]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.9849


0.9849295165099282

Model evaluation

In [20]:
import surprise
from surprise import evaluate

surprise.evaluate(algo, data, measures=[u'rmse', u'mae'], with_dump=False)

/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/lib/python3.6/dist-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9803
MAE:  0.7245
------------
Fold 2
RMSE: 0.9835
MAE:  0.7265
------------
Fold 3
RMSE: 0.9732
MAE:  0.7208
------------
Fold 4
RMSE: 0.9891
MAE:  0.7325
------------
Fold 5
RMSE: 0.9843
MAE:  0.7279
------------
------------
Mean RMSE: 0.9821
Mean MAE : 0.7265
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.7245479190305385,
                             0.7264969211089013,
                             0.7207892456081049,
                             0.7325185753055219,
                             0.7279133060656663],
                            'rmse': [0.9803356991355212,
                             0.9835127887875742,
                             0.9731966794625193,
                             0.9890701733346196,
                             0.9842836991906657]})